In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node
from tools import wait_for_node_finished


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_suggest',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_sugget = code

code_alm_sugget


In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
# g_dispfile_folder = nodebank.load("dispfile_folder")
# g_displace_result = nodebank.load("displace_result")
g_superstructure = nodebank.load("superstructure")  # 長周期安定構造

g_norder = nodebank.dump('cubic_norder', Int(2))  # cubic term
g_prefix = nodebank.dump('cubic_prefix', Str("cubic"))  # cubic term prefix
g_norder, g_prefix


In [ ]:
g_cutoff = Dict(dict={"*-*": {None, 7.3}})  # cubic term cutoff 定義

g_cutoff = nodebank.load_or_dump("cutoff2", g_cutoff)
g_cutoff


In [ ]:
g_primstructure = nodebank.load("primstructure")  # primitive cell
g_diag = nodebank.load("factor")  # 長周期
g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str, g_diag.get_array("factor").reshape(-1))),
                        g_prefix.value,
                        ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos


In [ ]:
g_root_wd = nodebank.load('root_wd')  # root directory
g_root_wd
g_cwd = Str(os.path.join(g_root_wd.value, g_wd_dos.value))
print(g_cwd)
g_cwd = nodebank.dump(f'{g_prefix.value}_cwd', g_cwd)  # cubic termの結果を置くsubdirectory
os.makedirs(g_cwd.value, exist_ok=True)


In [ ]:
#codename = "alm_suggest@tutor"
#code = Code.get_from_string(codename)
code = code_alm_sugget

builder = code.get_builder()
builder.structure = g_superstructure  # 長周期構造
builder.prefix = g_prefix  # 名前付け
# builder.mode = Str("suggest")
builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_alm_suggest"))  # 　保存directory
builder.norder = g_norder
builder.cutoff = g_cutoff

builder.metadata = {
    'options': {
        'resources': {'tot_num_mpiprocs': 1, 'num_machines': 1},
        'input_filename': "alm_suggest_cubic.in",
        'output_filename': "alm_suggest_cubic.out",
    }}

key = f"{g_prefix.value}_alm_suggest"

g_alm_suggest_future = nodebank.load(key, raise_error=False)
if g_alm_suggest_future is None:
    # if True:
    g_alm_suggest_future = submit(builder)  # temporary run(), use submit()
    print(g_alm_suggest_future)
    wait_for_node_finished(g_alm_suggest_future, 5)
    if g_alm_suggest_future.is_finished_ok:
        nodebank.dump(key, g_alm_suggest_future)


In [ ]:
nodebank.dump("{g_prefix.value}_suggest_pattern", g_alm_suggest_future.outputs.pattern)


In [ ]:
nodebank.dump(f"{g_prefix.value}_pattern",
              g_alm_suggest_future.outputs.pattern)

In [ ]:
g_alm_suggest_future.outputs.results.attributes


In [ ]:
g_cubic_pattern_files = g_alm_suggest_future.outputs.results["disp_pattern_filenames"]["ANHARM3"]
g_cubic_pattern_files = {'pattern_files': [g_cubic_pattern_files]}
g_cubic_pattern_files


In [ ]:
nodebank.dump(f"{g_prefix.value}_suggest_result", g_alm_suggest_future.outputs.results)
